## 1. Installing GEE

This is working with conda.\
For installing conda: https://developers.google.com/earth-engine/python_install-conda

Our conda install is managed by binder & configured using the `environment.yml`

#### Authenticate account (need it for the first time of use)

In [ ]:
# !earthengine authenticate

# NOTE: instead of running this in the notebook use the binder terminal
# 1. go to File > new > terminal
# 2. enter `earthengine authenticate` & follow prompts

In [2]:
cd /home/jovyan

/home/jovyan


In [24]:
# check to ensure the file we need is there
!ls -lh '/home/jovyan/.config/earthengine/credentials'

-rw------- 1 jovyan jovyan 124 Oct  8 18:40 /home/jovyan/.config/earthengine/credentials


## 2. Working with GEE

#### Import EE

In [ ]:
import ee
ee.Initialize()

In [20]:
## Verify the ee module is loaded.
print(ee.__version__)

0.1.283


#### Install other useful packages:

In [ ]:
pip install geemap ##Package for visualizing geospatial data.

#### Load satellite dataset: https://developers.google.com/earth-engine/datasets/

In [21]:
collection = ee.ImageCollection("COPERNICUS/S2_SR") ##Sentinel-2 L2A

#### Filter collection by date, region of interest and cloud coverage:

In [22]:
## Some settings:
start_date = '2020-01-01'
end_date = '2020-12-31'
cloud_cover = 30
states = ee.FeatureCollection("TIGER/2018/States") ## US administrative division
aoi = states.filterMetadata('NAME','equals','Florida') ## Select the state of Florida.

## Filter collection:
filtered = collection.filterDate(start_date,end_date) \
            .filterBounds(aoi) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover))

In [23]:
## Check number of images loaded:
print(filtered.size().getInfo())

2138


#### Clip collection and apply cloud mask:

In [ ]:
## Function to clip an image collection
def clipCollection(img):
    return img.clip(aoi)

## Function to mask clouds using the Sentinel-2 QA band.
def maskS2clouds(image):
    qa = image.select('QA60')

    ## Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;

    ## Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    ## Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000) \
    .select("B.*") \
    .copyProperties(image, ["system:time_start"])

In [ ]:
## Run functions and create mosaic:
mosaic = filtered.map(maskS2clouds).map(clipCollection).median()

#### Visualize map

In [ ]:
## Visualization parameters:
viz = {
    'min': 0,
    'max': 2.0,
    'bands': ['B4','B3','B2'],
    'gamma': 1.5}

# initialize our map
Map = geemap.Map()
Map.centerObject(aoi, 9)
Map.addLayer(mosaic, viz, "Florida")

Map.addLayerControl()
Map